In [ ]:
# item.py
import scrapy

class PttcrawlerItem(scrapy.Item):
    # define the fields for your item here like:
    # name = scrapy.Field()

    url = scrapy.Field()
    article_id = scrapy.Field()
    article_author = scrapy.Field()
    article_title = scrapy.Field()
    article_date = scrapy.Field()
    article_content = scrapy.Field()
    ip = scrapy.Field()
    message_count = scrapy.Field()
    messages = scrapy.Field()

In [ ]:
# settings.py

# Configure item pipelines
# See https://docs.scrapy.org/en/latest/topics/item-pipeline.html
ITEM_PIPELINES = {
#    'PTTcrawler.pipelines.PttcrawlerPipeline': 300,
    'PTTcrawler.pipelines.JSONPipeline':10
}

In [ ]:
# pipelines.py

import os
import json

from pathlib import Path
from datetime import datetime

class PttcrawlerPipeline(object):
    def process_item(self, item, spider):
        return item


class JSONPipeline(object):
    def open_spider(self, spider):
        self.start_crawl_datetime = datetime.now().strftime('%Y%m%dT%H:%M:%S')

        # 在開始爬蟲的時候建立暫時的 JSON 檔案
        # 避免有多筆爬蟲結果的時候，途中發生錯誤導致程式停止會遺失所有檔案
        self.dir_path = Path(__file__).resolve().parents[1] / 'crawled_data'
        self.runtime_file_path = str(self.dir_path / '.tmp.json.swp')
        if not self.dir_path.exists():
            self.dir_path.mkdir(parents=True)
        spider.log('Create temp file for store JSON - {}'.format(self.runtime_file_path))

        # 設計 JSON 存的格式為
        # [
        #  {...}, # 一筆爬蟲結果
        #  {...}, ...
        # ]
        self.runtime_file = open(self.runtime_file_path, 'w+', encoding='utf8')
        self.runtime_file.write('[\n')
        self._first_item = True

    def process_item(self, item, spider):
        # 把資料轉成字典格式並寫入文件中
        if not isinstance(item, dict):
            item = dict(item)

        if self._first_item:
            self._first_item = False
        else:
            self.runtime_file.write(',\n')

        self.runtime_file.write(json.dumps(item, ensure_ascii=False))
        return item

    def close_spider(self, spider):
        self.end_crawl_datetime = datetime.now().strftime('%Y%m%dT%H:%M:%S')

        # 儲存 JSON 格式
        self.runtime_file.write('\n]')
        self.runtime_file.close()
        
        # 將暫存檔改為以日期為檔名的格式
        self.store_file_path = self.dir_path / '{}-{}.json'.format(self.start_crawl_datetime,
                                                                   self.end_crawl_datetime)
        self.store_file_path = str(self.store_file_path)
        os.rename(self.runtime_file_path, self.store_file_path)
        spider.log('Save result at {}'.format(self.store_file_path))

In [ ]:
# PTT.py

import scrapy
from PTTcrawler.items import PttcrawlerItem
from bs4 import BeautifulSoup
from urllib.parse import urljoin, urlparse
from pathlib import Path
from pprint import pprint
import re


class PttSpider(scrapy.Spider):
    name = 'PTT'
    allowed_domains = ['www.ptt.cc']
    start_urls = ['https://www.ptt.cc/bbs/Examination/M.1641347124.A.630.html']
    cookies = {"over18": "1"}

    def start_requests(self):
        for url in self.start_urls:
            yield scrapy.Request(url=url, callback=self.parse, cookies=self.cookies)

    def parse(self, response):
        # 檢查網頁回應是否正確
        if response.status != 200:
            print('Error - {} is not available to access'.format(response.url))
            return
        
        # response 傳入 bs
        soup = BeautifulSoup(response.text)

        # 取得文章內容主體
        # 標題、作者、內容div id皆為main-content
        main_content = soup.find(id='main-content')

        # 假如文章有屬性資料 (meta), 我們在從屬性的區塊中爬出作者 (author), 文章標題 (title), 發文日期 (date)
        metas = main_content.select('div.article-metaline')
        author = ''
        title = ''
        date = ''
        if metas:
            if metas[0].select('span.article-meta-value')[0]:
                author = metas[0].select('span.article-meta-value')[0].string
            if metas[1].select('span.article-meta-value')[0]:
                title = metas[1].select('span.article-meta-value')[0].string
            if metas[2].select('span.article-meta-value')[0]:
                date = metas[2].select('span.article-meta-value')[0].string

            # 提取完作者、標題、日期後，移除掉
            # .extract() 方法可以參考官方文件
            #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#extract
            for m in metas:
                m.extract()
            for m in main_content.select('div.article-metaline-right'):
                m.extract()

        # 取得留言區主體
        pushes = main_content.find_all('div', class_='push')
        for p in pushes:
            p.extract()

        # 假如文章中有包含「※ 發信站: 批踢踢實業坊(ptt.cc), 來自: xxx.xxx.xxx.xxx」的樣式
        # 透過 regular expression 取得 IP
        # 因為字串中包含特殊符號跟中文, 這邊建議使用 unicode 的型式 u'...'
        try:
            ip = main_content.find(text=re.compile(u'※ 發信站:'))
            ip = re.search('[0-9]*\.[0-9]*\.[0-9]*\.[0-9]*', ip).group()
        except Exception as e:
            ip = ''

        # 移除文章主體中 '※ 發信站:', '◆ From:', 空行及多餘空白 (※ = u'\u203b', ◆ = u'\u25c6')
        # 保留英數字, 中文及中文標點, 網址, 部分特殊符號
        #
        # 透過 .stripped_strings 的方式可以快速移除多餘空白並取出文字, 可參考官方文件 
        #  - https://www.crummy.com/software/BeautifulSoup/bs4/doc/#strings-and-stripped-strings
        filtered = []
        for v in main_content.stripped_strings:
            # 假如字串開頭不是特殊符號或是以 '--' 開頭的, 我們都保留其文字
            if v[0] not in [u'※', u'◆'] and v[:2] not in [u'--']:
                filtered.append(v)

        
        # 定義一些特殊符號與全形符號的過濾器
        expr = re.compile(u'[^一-龥。；，：“”（）、？《》\s\w:/-_.?~%()]')
        for i in range(len(filtered)):
            filtered[i] = re.sub(expr, '', filtered[i])

        # 移除空白字串, 組合過濾後的文字即為文章本文 (content)
        filtered = [i for i in filtered if i]
        content = ' '.join(filtered)


        # 處理留言區
        # p 計算推文數量
        # b 計算噓文數量
        # n 計算箭頭數量
        p, b, n = 0, 0, 0
        messages = []
        for push in pushes:
            # 假如留言段落沒有 push-tag 就跳過
            if not push.find('span', 'push-tag'):
                continue

            # 過濾額外空白與換行符號
            # push_tag 判斷是推文, 箭頭還是噓文
            # push_userid 判斷留言的人是誰
            # push_content 判斷留言內容
            # push_ipdatetime 判斷留言日期時間
            push_tag = push.find('span', 'push-tag').string.strip(' \t\n\r')
            push_userid = push.find('span', 'push-userid').string.strip(' \t\n\r')
            push_content = push.find('span', 'push-content').strings
            push_content = ' '.join(push_content)[1:].strip(' \t\n\r')
            push_ipdatetime = push.find('span', 'push-ipdatetime').string.strip(' \t\n\r')

            # 整理打包留言的資訊, 並統計推噓文數量
            messages.append({
                'push_tag': push_tag,
                'push_userid': push_userid,
                'push_content': push_content,
                'push_ipdatetime': push_ipdatetime})
            if push_tag == u'推':
                p += 1
            elif push_tag == u'噓':
                b += 1
            else:
                n += 1

        # 統計推噓文
        # count 為推噓文相抵看這篇文章推文還是噓文比較多
        # all 為總共留言數量 
        message_count = {'all': p+b+n, 'count': p-b, 'push': p, 'boo': b, 'neutral': n}

        # 整理文章資訊
        data = PttcrawlerItem()
        article_id = str(Path(urlparse(response.url).path).stem)
        data['url'] = response.url
        data['article_id'] = article_id
        data['article_author'] = author
        data['article_title'] = title
        data['article_date'] = date
        data['article_content'] = content
        data['ip'] = ip
        data['message_count'] = message_count
        data['messages'] = messages
        yield data

In [1]:
# 存好的JSON

import json
with open("./Data/PTT_crawler/crawled_data/.tmp.json.swp", encoding="utf-8") as j:
    data = json.load(j)
print(type(data))
print(type(data[0]))

print(data[0]['url'])
print(data[0]['article_id'])
print("--------------------")
print(data[0]['article_author'])
print(data[0]['article_title'])
print(data[0]['article_date'])
print("--------------------")
print(data[0]['article_content'])
print("IP：", data[0]['ip'])
print("總留言數:{}".format(data[0]['message_count']['all']))
print("推噓差:{}".format(data[0]['message_count']['count']))
print("推文數:{}".format(data[0]['message_count']['push']))
print("噓文數:{}".format(data[0]['message_count']['boo']))
print("箭頭數:{}".format(data[0]['message_count']['neutral']))
print("--------------------")
for i in data[0]['messages']:
    print(i['push_tag'], i['push_userid'], i['push_content'], i['push_ipdatetime'])

<class 'list'>
<class 'dict'>
https://www.ptt.cc/bbs/Examination/M.1641347124.A.630.html
M.1641347124.A.630
--------------------
gary22204 ()
[閒聊] 110高考資訊處理，如果我考第16名怎麼辦?
Wed Jan  5 09:45:22 2022
--------------------
安安我上榜學長啦

昨天中午跟公家好同事一起出去運動，想起是放榜日

畢竟我們單位今年好像也有缺，就看了一下榜單

不看得了，一看嚇死我的寶寶

資訊處理高考居然只有15個正額錄取

上榜學長我尋思，這年頭資訊爆炸，資訊缺只會愈來愈多，顯然不正常

這時候...(柯南音樂下)...真相只有一個


於是，看了最新的暫定需用名額統計表，居然有80個

80葛AAAAAAAAAAAAAAAAAA

但4只有15個人入取AAAAAAAAAAAAAAAAAA


所以有兩個可能：1.考題太難大家都沒到50分    2.閱卷的人太雕

真相果然只有一葛



於是，學長我就想了一下

如果我含辛茹苦全職讀了一整年兩整年

好不容易考了全國前16名，準備谷底反轉，泥封高輝

但4，好像落榜了AAAAAAAAAAAAA
ob_ov


又，資訊單位在這疫情這下死撐活撐2年，考試又延了快半年

終於迎來新人補進的時機了

結果，國家一整年的補進資訊人員的機會，居然因為出題過難

就這樣嚴重拖慢國家的效率




身為103年的學長我只有一個感想

拖慢考生拖慢機構，真不愧是我大鬼島台灣



請教今年有考試的考生們，到底是閱卷太嚴苛

還是人家所謂的年輕人不夠努力，一代不如一代?ob~~~ov

 https://www.ptt.cc/bbs/Examination/M.1641347124.A.630.html
IP： 117.56.223.223
總留言數:26
推噓差:13
推文數:14
噓文數:1
箭頭數:11
--------------------
推 carterdunk 天下無難事，只要有新人。 01/05 09:48
→ carterdunk 但沒有新人怎麼辦？ 01/05 09:49
→ vv956 標準在那邊，依法行政～ 01/05 0